In [1]:
import kagglehub
import pandas as pd
import numpy as np

path_pop = kagglehub.dataset_download("iamsouravbanerjee/world-population-dataset")
df_pop = pd.read_csv(f"{path_pop}/world_population.csv")

# 2. Annual CO2 Emissions by Country
path_co2 = kagglehub.dataset_download("ulrikthygepedersen/co2-emissions-by-country")
df_co2 = pd.read_csv(f"{path_co2}/co2_emissions_kt_by_country.csv")

# 3. Monthly Climate Data by Station
path_station = kagglehub.dataset_download("christopherlemke/monthly-climat-reports-from-stations-worldwide")
df_reports = pd.read_csv(f"{path_station}/dwd-cdc_CLIMAT_reports_stations_ww.csv")
df_stations = pd.read_csv(f"{path_station}/dwd-cdc_station_data_ww.csv")

path_aq1 = kagglehub.dataset_download("kanchana1990/world-air-quality-data-2024-updated")
df_aq1 = pd.read_csv(f"{path_aq1}/world_air_quality.csv", on_bad_lines='skip', sep=';')

path_aq2 = kagglehub.dataset_download("dnkumars/air-quality-index")
df_aq2 = pd.read_csv(f"{path_aq2}/cleaned_interpolated_dataset.csv")

/home/amcar/Desktop/climate_lens/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Co2 DF

In [2]:
co2_countries = df_co2['country_code'].unique()
pop_countries = df_pop['CCA3'].unique()

In [3]:
country_map = df_pop.loc[:, ['CCA3', 'Country/Territory']].set_index('CCA3')['Country/Territory']
country_map.to_csv('country_map.csv')
country_map

CCA3
AFG          Afghanistan
ALB              Albania
DZA              Algeria
ASM       American Samoa
AND              Andorra
             ...        
WLF    Wallis and Futuna
ESH       Western Sahara
YEM                Yemen
ZMB               Zambia
ZWE             Zimbabwe
Name: Country/Territory, Length: 234, dtype: object

In [4]:
missing_countries = set(co2_countries) - set(pop_countries)
df_co2[df_co2['country_code'].isin(missing_countries)]['country_name'].unique()


array(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Central Europe and the Baltics',
       'Caribbean small states',
       'East Asia & Pacific (excluding high income)',
       'Early-demographic dividend', 'East Asia & Pacific',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia', 'Euro area', 'European Union',
       'Fragile and conflict affected situations', 'High income',
       'Heavily indebted poor countries (HIPC)', 'IBRD only',
       'IDA & IBRD total', 'IDA total', 'IDA blend', 'IDA only',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & Caribbean',
       'Least developed countries: UN classification', 'Low income',
       'Lower middle income', 'Low & middle income',
       'Late-demographic dividend', 'Middle East & North Africa',
       'Middle income',
       'Middle East & North Africa (excluding high income)',
       'North America', 'OECD members', 'Other sm

In [5]:

df_co2 = df_co2[~df_co2['country_code'].isin(missing_countries)]


In [6]:
# Step 1: Prepare df_pop in long format
df_pop_long = df_pop.melt(
    id_vars=['CCA3'],
    value_vars=['1970 Population', '1980 Population', '1990 Population', '2000 Population',
                '2010 Population', '2015 Population', '2020 Population', '2022 Population'],
    var_name='year',
    value_name='population'
)

# Clean and convert year to integer
df_pop_long['year'] = df_pop_long['year'].str.extract(r'(\d+)').astype(int)

all_years = np.arange(df_pop_long['year'].min(), df_pop_long['year'].max() + 1)

df_list = []
for country, group in df_pop_long.groupby('CCA3'):
    group = group.set_index('year').reindex(all_years)
    group['CCA3'] = country
    group['population'] = group['population'].interpolate(method='linear')
    df_list.append(group.reset_index())

df_pop_interp = pd.concat(df_list, ignore_index=True)

df_pop_interp

,year,CCA3,population
0,1970,ABW,59106.0
1,1971,ABW,59422.1
2,1972,ABW,59738.2
3,1973,ABW,60054.3
4,1974,ABW,60370.4
...,...,...,...
12397,2018,ZWE,15063774.4
12398,2019,ZWE,15366720.2
12399,2020,ZWE,15669666.0
12400,2021,ZWE,15995101.5


In [7]:
# Step 3: Merge safely
df_co2 = df_co2.merge(
    df_pop_interp,
    how='inner',
    left_on=['country_code', 'year'],
    right_on=['CCA3', 'year']
).drop(columns=['CCA3'])


In [8]:
df_co2['co2_per_capita'] = df_co2['value'] / df_co2['population']
df_co2 = df_co2.drop(columns=["country_name", "population"])
df_co2.rename(columns={'value':'co2'})

df_co2.to_csv('co2.csv')
print(df_co2.isna().sum())
df_co2.head()

country_code      0
year              0
value             0
co2_per_capita    0
dtype: int64


,country_code,year,value,co2_per_capita
0,ABW,1970,16655.514,0.281791
1,ABW,1971,14495.651,0.243944
2,ABW,1972,14055.611,0.235287
3,ABW,1973,15592.084,0.259633
4,ABW,1974,14132.618,0.234098


## Climate DF

In [9]:
# --- Step 0: Rename station columns ---
df_stations = df_stations.rename(columns={
    '0': 'id',
    '1': 'name',
    '2': 'latitude',
    '3': 'longitude',
    '4': 'height',
    '5': 'country'
})

# --- Step 1: Keep only rows with non-missing, numeric IDs ---
df_reports = df_reports[pd.to_numeric(df_reports["IIiii"], errors='coerce').notna()].copy()
df_stations = df_stations[pd.to_numeric(df_stations["id"], errors='coerce').notna()].copy()

# --- Step 2: Convert to int → str ---
df_reports["IIiii"] = df_reports["IIiii"].astype(float).astype(int).astype(str)
df_stations["id"] = df_stations["id"].astype(float).astype(int).astype(str)

# --- Step 3: Check overlap ---
reports_ids = set(df_reports["IIiii"])
stations_ids = set(df_stations["id"])
overlap_ids = reports_ids & stations_ids
print(f"Overlapping IDs: {len(overlap_ids)}")

# --- Step 4: Merge using inner join ---
df_climate = pd.merge(
    df_reports,
    df_stations,
    left_on="IIiii",
    right_on="id",
    how="inner"
).drop(columns=['id'])

print(df_climate.head())



Overlapping IDs: 3458
     year  month IIiii   G1       Po  G1.1        P  G1.2   sn     T  ...  \
0  2013.0    4.0  1001  1.0  10095.0   2.0  10107.0   3.0  1.0  42.0  ...   
1  2013.0    4.0  1007  1.0  10084.0   2.0  10094.0   3.0  1.0  92.0  ...   
2  2013.0    4.0  1008  1.0  10056.0   2.0  10091.0   3.0  1.0  93.0  ...   
3  2013.0    4.0  1025  1.0  10043.0   2.0  10054.0   3.0  0.0  10.0  ...   
4  2013.0    4.0  1026  1.0   9912.0   2.0  10054.0   3.0  0.0  12.0  ...   

   Dgr  G4.7  iy  Gx  Gn                      name  \
0  NaN   NaN NaN NaN NaN                 Jan Mayen   
1  NaN   NaN NaN NaN NaN                Ny-Alesund   
2  NaN   NaN NaN NaN NaN                  Svalbard   
3  NaN   NaN NaN NaN NaN            Tromso/Langnes   
4  NaN   NaN NaN NaN NaN                    Tromso   

                                        latitude   longitude   height  \
0                                          70.94      -08.67        9   
1                                          7

In [10]:
import pandas as pd

# Clean country names
df_climate['country'] = df_climate['country'].str.replace('\r\n', '').str.strip()


df_climate = df_climate.loc[:, ['country', 'year', 'month','sn', 'T', 'R1', 'sn.1', 'Tx', 'sn.2', 'Tn']]
df_climate['temp_mean'] = (1 - 2 * df_climate['sn']) * df_climate['T'] / 10
df_climate['temp_max']  = (1 - 2 * df_climate['sn.1']) * df_climate['Tx'] / 10
df_climate['temp_min']  = (1 - 2 * df_climate['sn.2']) * df_climate['Tn']  / 10# assuming sn.2 also applies to Tn


# Step 1: Compute monthly mean for each year-country-month
df_climate = (
    df_climate
    .groupby(['year', 'country', 'month'], as_index=False)
    .mean()  # averages sn, T, R1, temp
)

# Compute signed temperatures

# Optional: drop original columns if no longer needed
df_climate = df_climate.drop(columns=['T', 'Tx', 'Tn', 'sn', 'sn.1', 'sn.2'])

df_climate['year'] = df_climate['year'].astype(int)
df_climate['month'] = df_climate['month'].astype(int)
# Check result
df_climate.head()


,year,country,month,R1,temp_mean,temp_max,temp_min
0,2003,Algeria,3,16.200000,15.023529,20.605882,9.917647
1,2003,Algeria,4,1025.634146,18.173171,24.068293,12.251220
2,2003,Algeria,5,545.631579,21.707018,27.944643,15.366667
3,2003,Algeria,6,1760.052632,27.694828,34.146552,21.162069
4,2003,Algeria,7,447.311111,30.135556,36.644444,23.735556


In [11]:
print(set(df_climate['country'].unique()) - set(country_map.unique()))
print(set(country_map.unique()) - set(df_climate['country'].unique()))

{'', 'Western-Sahara', 'Palau-Islands', 'Indonesien', "People's Dem. Rep. Laos", 'Marshall-Islands', 'Moldova, Rep. Of', 'Australien, SW-Pazifik', 'Wallis-Islands', 'Dem. Republic of the Congo', 'Macedonia', 'Syrian Arab Rep.', 'Caroline-Islands', 'Slovakia (Slovak. Rep.)', 'St. Maarten', 'Wake-Insel', 'Bosnia and Herzegowina', 'Republic of Korea', 'Republic of China, Taiwan', 'United States of America', 'Iran (Islamic Rep. of)', 'Ascencion Island', 'Tunesia', 'Croatia/Hrvatska', 'United Kingdom of Great Britain and N.-Ireland', "Korea, Dem. People's Rep.", 'Russian Federation', 'Cook-Island', 'Slowenia', 'United Arab. Emirates', 'Mauretania', "Cote d'Ivoire"}
{'Micronesia', 'Somalia', 'Lesotho', 'Croatia', 'Estonia', 'Marshall Islands', 'Cook Islands', 'Liechtenstein', 'Saint Barthelemy', 'Eswatini', 'Antigua and Barbuda', 'Mayotte', 'Eritrea', 'Bhutan', 'United States Virgin Islands', 'Wallis and Futuna', 'Guatemala', 'Sierra Leone', 'Timor-Leste', 'North Korea', 'Djibouti', 'Monaco'

In [12]:
country_corrections = {
    'Australien, SW-Pazifik': 'Australia',
    'Cook-Island': 'Cook Islands',
    'United States of America': 'United States',
    'Caroline-Islands': 'Micronesia',
    'Tunesia': 'Tunisia',
    'St. Maarten': 'Sint Maarten',
    'Croatia/Hrvatska': 'Croatia',
    'Republic of China, Taiwan': 'Taiwan',
    'Slowenia': 'Slovenia',
    'Indonesien': 'Indonesia',
    'Palau-Islands': 'Palau',
    "Cote d'Ivoire": 'Ivory Coast',
    'Russian Federation': 'Russia',
    "Korea, Dem. People's Rep.": 'North Korea',
    'Iran (Islamic Rep. of)': 'Iran',
    'Mauretania': 'Mauritania',
    'Marshall-Islands': 'Marshall Islands',
    'Macedonia': 'North Macedonia',
    'United Arab. Emirates': 'United Arab Emirates',
    'Republic of Korea': 'South Korea',
    'United Kingdom of Great Britain and N.-Ireland': 'United Kingdom',
    "People's Dem. Rep. Laos": 'Laos',
    'Bosnia and Herzegowina': 'Bosnia and Herzegovina',
    'Ascencion Island': 'Ascension Island',
    'Syrian Arab Rep.': 'Syria',
    'Dem. Republic of the Congo': 'DR Congo',
    'Slovakia (Slovak. Rep.)': 'Slovakia',
    'Western-Sahara': 'Western Sahara',
    'Wake-Insel': 'Wake Island',
    'Moldova, Rep. Of': 'Moldova',
    'Wallis-Islands': 'Wallis and Futuna',
    '': None  # optional: empty string to None
}
df_climate['country'] = df_climate['country'].replace(country_corrections)

# country_map: index = code, values = country_name
# Create a reversed mapping: country_name -> code
reversed_map = pd.Series(country_map.index, index=country_map.values)

# Now map the country names to codes
df_climate['country_code'] = df_climate['country'].map(reversed_map)
print(f"NA codes: {df_climate.isna().sum()}")

df_climate = df_climate.drop(columns = 'country')
df_climate = df_climate.dropna(subset=["country_code", "temp_mean"])

df_climate.to_csv('climate.csv')
df_climate.head()

NA codes: year               0
country          159
month              0
R1               210
temp_mean        328
temp_max        1141
temp_min        1151
country_code     392
dtype: int64


,year,month,R1,temp_mean,temp_max,temp_min,country_code
0,2003,3,16.200000,15.023529,20.605882,9.917647,DZA
1,2003,4,1025.634146,18.173171,24.068293,12.251220,DZA
2,2003,5,545.631579,21.707018,27.944643,15.366667,DZA
3,2003,6,1760.052632,27.694828,34.146552,21.162069,DZA
4,2003,7,447.311111,30.135556,36.644444,23.735556,DZA


## AQI DFs

### AQ1: Various Pollutants

In [13]:
df_aq1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54255 entries, 0 to 54254
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Code   54255 non-null  object 
 1   City           30209 non-null  object 
 2   Location       54253 non-null  object 
 3   Coordinates    54185 non-null  object 
 4   Pollutant      54255 non-null  object 
 5   Source Name    54255 non-null  object 
 6   Unit           54255 non-null  object 
 7   Value          54255 non-null  float64
 8   Last Updated   54255 non-null  object 
 9   Country Label  54140 non-null  object 
dtypes: float64(1), object(9)
memory usage: 4.1+ MB


In [14]:
units = df_aq1.groupby('Pollutant')['Unit'].agg(lambda x: x.value_counts().idxmax())

df_aq1 = df_aq1[df_aq1['Unit'] == df_aq1['Pollutant'].map(units)]

means_by_country = df_aq1.groupby(['Country Label', 'Pollutant', 'Unit'])['Value'].mean()
# Collapse the MultiIndex into columns
means_by_country = means_by_country.reset_index()



In [15]:
means_by_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Label  487 non-null    object 
 1   Pollutant      487 non-null    object 
 2   Unit           487 non-null    object 
 3   Value          487 non-null    float64
dtypes: float64(1), object(3)
memory usage: 15.3+ KB


In [16]:
country_label_corrections = {
    'Russian Federation': 'Russia',
    "Lao People's Dem. Rep.": 'Laos',
    'Sudan, The Republic of': 'Sudan',
    'Congo, Democratic Republic of the': 'DR Congo',
    'Korea, Republic of': 'South Korea',
    'Viet Nam': 'Vietnam',
    'Moldova, Republic of': 'Moldova',
    "Côte d'Ivoire": 'Ivory Coast',
    'Hong Kong, China': 'Hong Kong',
    'Taiwan, China': 'Taiwan',
    'USSR': 'Russia',  # historical mapping
    'Serbia and Montenegro': 'Serbia',
    'Macedonia, The former Yugoslav Rep. of': 'North Macedonia'
}
means_by_country['Country Label'] = means_by_country['Country Label'].replace(country_label_corrections)

# Map country names to codes
means_by_country['country_code'] = means_by_country['Country Label'].map(reversed_map)

In [17]:
means_by_country.isna().sum()

Country Label    0
Pollutant        0
Unit             0
Value            0
country_code     0
dtype: int64

In [18]:
means_by_country.drop(columns=['Country Label'], inplace=True)
means_by_country.rename(columns={
    'Value':'value',
    'Pollutant':'pollutant',
    'Unit':'unit'}, inplace=True)

means_by_country.to_csv('pollution.csv')

### AQ2: AQI by country by Month

In [19]:
cols = [c for c in df_aq2.columns if c not in ['rank', 'city', 'city_name', 'Country']]
df_aq = df_aq2.groupby('Country')[cols].mean()

df_aq.to_csv('air_quality.csv')
df_aq.sort_values('avg', ascending=False).head()

,avg,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
Country,,,,,,,,,,,,,
Bangladesh,161.000000,303.000000,195.000000,184.000000,116.000000,125.000000,100.000000,51.000000,107.000000,83.000000,158.000000,223.000000,284.000000
Chad,147.000000,221.000000,326.000000,189.000000,285.000000,106.000000,87.500000,58.500000,38.000000,35.000000,207.000000,228.000000,235.000000
Kuwait,95.333333,90.000000,96.000000,72.666667,78.333333,111.333333,117.666667,92.333333,110.000000,108.666667,97.666667,81.333333,86.666667
India,83.283372,132.065574,109.847775,85.391101,77.925059,67.594848,62.161593,42.587822,55.494145,48.189696,81.800937,114.119438,121.674473
Pakistan,79.166667,124.500000,82.333333,61.833333,42.666667,36.666667,41.833333,33.000000,41.222222,45.555556,77.666667,177.500000,191.666667


In [20]:
df_aq = df_aq.reset_index()


In [21]:
# Countries in df_aq not in country_map
print("Countries in df_aq not in country_map:")
print(set(df_aq['Country'].unique()) - set(country_map.unique()))

# Countries in country_map not in df_aq
print("Countries in country_map not in df_aq:")
print(set(country_map.unique()) - set(df_aq['Country'].unique()))


Countries in df_aq not in country_map:
{'Cabo Verde', 'Congo (Kinshasa)', 'Bosnia And Herzegovina', 'Macedonia', 'Trinidad And Tobago', 'Cote D’Ivoire', 'Burma', 'Czechia', 'Kosovo'}
Countries in country_map not in df_aq:
{'Micronesia', 'Suriname', 'Somalia', 'Papua New Guinea', 'Tuvalu', 'Lesotho', 'Kiribati', 'Central African Republic', 'Cook Islands', 'Marshall Islands', 'Myanmar', 'American Samoa', 'Comoros', 'Saint Barthelemy', 'Eswatini', 'Bermuda', 'Antigua and Barbuda', 'Mayotte', 'Eritrea', 'Fiji', 'Dominican Republic', 'Bhutan', 'United States Virgin Islands', 'Wallis and Futuna', 'Uruguay', 'Sierra Leone', 'Maldives', 'Tanzania', 'Timor-Leste', 'Djibouti', 'Paraguay', 'Samoa', 'Greenland', 'Oman', 'Bahamas', 'Niue', 'Equatorial Guinea', 'Mozambique', 'Sint Maarten', 'Faroe Islands', 'Panama', 'Mauritius', 'DR Congo', 'Hong Kong', 'Vatican City', 'Vanuatu', 'French Polynesia', 'Nauru', 'Nicaragua', 'Dominica', 'Republic of the Congo', 'Tokelau', 'Saint Martin', 'Niger', 'West

In [22]:
country_fix_aq = {
    'Czechia': 'Czech Republic',
    'Cabo Verde': 'Cape Verde',
    'Trinidad And Tobago': 'Trinidad and Tobago',
    'Cote D’Ivoire': 'Ivory Coast',
    'Congo (Kinshasa)': 'DR Congo',
    'Kosovo': 'XK',  # Kosovo may not be in country_map
    'Burma': 'Myanmar',
    'Bosnia And Herzegovina': 'Bosnia and Herzegovina',
    'Macedonia': 'North Macedonia'
}
df_aq['Country'] = df_aq['Country'].replace(country_fix_aq)
df_aq['country_code'] = df_aq['Country'].map(reversed_map)

In [23]:
df_aq.isna().sum()

Country         0
avg             0
jan             0
feb             0
mar             0
apr             0
may             0
jun             0
jul             0
aug             0
sep             0
oct             0
nov             0
dec             0
country_code    1
dtype: int64

In [25]:
df_aq.dropna(inplace=True)
df_aq.drop(columns=['Country', ], inplace=True)

In [26]:
df_aq.to_csv('air_quality.csv')

## Imputation of countries

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Load all datasets
air_quality = pd.read_csv('data/air_quality.csv')
climate = pd.read_csv('data/climate.csv')
co2 = pd.read_csv('data/co2.csv')
pollution = pd.read_csv('data/pollution.csv')

print("Loaded datasets:")
print(f"Air Quality: {air_quality.shape}")
print(f"Climate: {climate.shape}")
print(f"CO2: {co2.shape}")
print(f"Pollution: {pollution.shape}")

# Prepare pollution data - pivot to have pollutants as columns
pollution_pivot = pollution.pivot_table(
    index='country_code', 
    columns='pollutant', 
    values='value', 
    aggfunc='mean'
).reset_index()

# Merge datasets on country_code
# Start with air_quality as base
merged = air_quality.copy()

# Aggregate climate data by country (average across all years/months)
climate_agg = climate.groupby('country_code').agg({
    'R1': 'mean',
    'temp_mean': 'mean',
    'temp_max': 'mean',
    'temp_min': 'mean'
}).reset_index()

# Aggregate CO2 data by country (average across all years)
co2_agg = co2.groupby('country_code').agg({
    'value': 'mean',
    'co2_per_capita': 'mean'
}).rename(columns={'value': 'co2_mean'}).reset_index()

# Merge all datasets
merged = merged.merge(climate_agg, on='country_code', how='inner')
merged = merged.merge(co2_agg, on='country_code', how='inner')
merged = merged.merge(pollution_pivot, on='country_code', how='inner')

print(f"\nMerged dataset shape: {merged.shape}")
print(f"\nMissing values in merged dataset:")
print(merged.isna().sum().sort_values(ascending=False))

# Select columns with numeric data for imputation
numeric_cols = merged.select_dtypes(include=[np.number]).columns.tolist()
# Remove country_code if it's numeric (shouldn't be used as feature)
if 'country_code' in numeric_cols:
    numeric_cols.remove('country_code')

# ============================================================================
# CORRELATION ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

# Calculate correlation matrix for numeric columns
corr_data = merged[numeric_cols].copy()

# Calculate pairwise correlations
correlation_matrix = corr_data.corr()

print(f"\nCorrelation matrix shape: {correlation_matrix.shape}")
print("\nTop correlations (absolute value > 0.5):")
print("-" * 80)

# Find top correlations (excluding self-correlations)
top_correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        col1 = correlation_matrix.columns[i]
        col2 = correlation_matrix.columns[j]
        corr_val = correlation_matrix.iloc[i, j]
        if not np.isnan(corr_val) and abs(corr_val) > 0.5:
            top_correlations.append((col1, col2, corr_val))

# Sort by absolute correlation value
top_correlations.sort(key=lambda x: abs(x[2]), reverse=True)

# Display top 10 correlations
for col1, col2, corr_val in top_correlations[:10]:
    print(f"{col1:25s} <-> {col2:25s} : {corr_val:7.4f}")

if len(top_correlations) == 0:
    print("No strong correlations found (|r| > 0.5)")

# Create a small test dataset (select first 50 rows with sufficient non-missing data)
test_data = merged.dropna(subset=numeric_cols[:5]).head(50).copy()
print(f"\nTest dataset shape: {test_data.shape}")

# Function to perform correlation-based imputation
def impute_correlation(data, target_col, test_ratio=0.3, random_state=None):
    """
    Artificially introduce missing values, impute them using correlation with most correlated variable, and evaluate
    Returns: (mse, mae) for correlation-based imputation
    """
    np.random.seed(random_state)
    
    # Copy data
    test_df = data.copy()
    
    # Select rows where target column is not missing
    valid_mask = ~test_df[target_col].isna()
    valid_indices = test_df[valid_mask].index
    
    if len(valid_indices) == 0:
        return None, None
    
    # Randomly select indices to make missing
    n_missing = max(1, int(len(valid_indices) * test_ratio))
    missing_indices = np.random.choice(valid_indices, n_missing, replace=False)
    
    # Store true values
    true_values = test_df.loc[missing_indices, target_col].copy()
    
    # Make values missing
    test_df.loc[missing_indices, target_col] = np.nan
    
    # Prepare features (all other numeric columns)
    feature_cols = [col for col in numeric_cols if col != target_col]
    
    if len(feature_cols) == 0:
        return None, None
    
    # Work with data, fill missing features with median
    test_df_imputed = test_df.copy()
    for col in feature_cols:
        median_val = test_df[col].median()
        if pd.notna(median_val):
            test_df_imputed[col] = test_df_imputed[col].fillna(median_val)
    
    # Get training data (where target is not missing)
    train_mask = ~test_df_imputed[target_col].isna()
    train_data = test_df_imputed[train_mask].copy()
    
    # Get test data (where target is missing)
    test_missing = test_df_imputed[test_df_imputed.index.isin(missing_indices)].copy()
    
    if len(train_data) < 2 or len(test_missing) == 0:
        return None, None
    
    # Calculate correlation between target and all features in training data
    correlations = {}
    for col in feature_cols:
        if col in train_data.columns:
            # Get rows where both target and feature are not NaN
            both_notna = train_data[[target_col, col]].dropna()
            if len(both_notna) >= 2:
                corr = both_notna[target_col].corr(both_notna[col])
                if pd.notna(corr):
                    correlations[col] = abs(corr)
    
    if len(correlations) == 0:
        return None, None
    
    # Find the most correlated feature
    most_correlated = max(correlations, key=correlations.get)
    
    # Use linear relationship between target and most correlated feature
    # Fit simple linear regression: target = a * feature + b
    both_notna = train_data[[target_col, most_correlated]].dropna()
    if len(both_notna) < 2:
        return None, None
    
    # Calculate slope and intercept
    x = both_notna[most_correlated].values
    y = both_notna[target_col].values
    
    # Simple linear regression: y = ax + b
    slope = np.cov(x, y)[0, 1] / np.var(x) if np.var(x) > 0 else 0
    intercept = np.mean(y) - slope * np.mean(x)
    
    # Impute missing values using: target = slope * feature + intercept
    test_x = test_missing[most_correlated].values
    predicted = slope * test_x + intercept
    
    # Get true values
    true_vals_list = []
    valid_indices_list = []
    
    for i, idx in enumerate(test_missing.index):
        if idx in true_values.index:
            true_vals_list.append(true_values.loc[idx])
            valid_indices_list.append(i)
    
    if len(true_vals_list) == 0:
        return None, None
    
    true_vals = np.array(true_vals_list)
    predicted = predicted[valid_indices_list]
    
    # Calculate metrics
    if len(predicted) > 0 and len(true_vals) > 0:
        mse = mean_squared_error(true_vals, predicted)
        mae = mean_absolute_error(true_vals, predicted)
        return mse, mae
    
    return None, None

# Function to perform imputation and evaluation with regression analysis
def impute_and_evaluate(data, target_col, test_ratio=0.3, random_state=None):
    """
    Artificially introduce missing values, impute them using regression analysis, and evaluate
    Returns: (mse, mae, regression_stats) where regression_stats includes R², coefficients, etc.
    """
    from sklearn.metrics import r2_score
    
    np.random.seed(random_state)
    
    # Copy data
    test_df = data.copy()
    
    # Select rows where target column is not missing
    valid_mask = ~test_df[target_col].isna()
    valid_indices = test_df[valid_mask].index
    
    if len(valid_indices) == 0:
        return None, None, None
    
    # Randomly select indices to make missing
    n_missing = max(1, int(len(valid_indices) * test_ratio))
    missing_indices = np.random.choice(valid_indices, n_missing, replace=False)
    
    # Store true values
    true_values = test_df.loc[missing_indices, target_col].copy()
    
    # Make values missing
    test_df.loc[missing_indices, target_col] = np.nan
    
    # Prepare features (all other numeric columns)
    feature_cols = [col for col in numeric_cols if col != target_col]
    
    if len(feature_cols) == 0:
        return None, None, None
    
    # Work with all data, but fill missing features with median
    # This allows us to use more data points
    test_df_imputed = test_df.copy()
    
    # For each feature column, fill missing values with median
    for col in feature_cols:
        median_val = test_df[col].median()
        if pd.notna(median_val):
            test_df_imputed[col] = test_df_imputed[col].fillna(median_val)
    
    # Separate into train (with target) and test (missing target)
    # Train: rows where target is NOT missing
    train_mask = ~test_df_imputed[target_col].isna()
    train_data = test_df_imputed[train_mask].copy()
    
    # Test: rows where target IS missing (the ones we artificially made missing)
    test_missing = test_df_imputed[test_df_imputed.index.isin(missing_indices)].copy()
    
    # Ensure we have enough training data and test data
    if len(train_data) < 2:
        return None, None, None
    
    if len(test_missing) == 0:
        # If test_missing is empty, try to find any rows with missing target
        test_missing = test_df_imputed[test_df_imputed[target_col].isna()].copy()
        if len(test_missing) == 0:
            return None, None, None
    
    # Prepare X and y (features are already imputed, but ensure no remaining NaN)
    X_train = train_data[feature_cols].copy()
    X_test = test_missing[feature_cols].copy()
    
    # Remove columns that are entirely NaN
    valid_features = []
    for col in feature_cols:
        if col in X_train.columns:
            # Check if column has any valid values in training data
            if X_train[col].notna().sum() > 0:
                valid_features.append(col)
    
    if len(valid_features) == 0:
        return None, None, None
    
    X_train = X_train[valid_features].copy()
    X_test = X_test[valid_features].copy()
    
    # Fill remaining NaN values with column median, or 0 if median is NaN
    for col in valid_features:
        median_val = X_train[col].median()
        if pd.isna(median_val):
            # If median is NaN, use mean, or 0 if that's also NaN
            mean_val = X_train[col].mean()
            fill_val = mean_val if pd.notna(mean_val) else 0.0
        else:
            fill_val = median_val
        
        X_train[col] = X_train[col].fillna(fill_val)
        X_test[col] = X_test[col].fillna(fill_val)
    
    # Final check: replace any remaining NaN with 0
    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)
    
    # Convert to numpy arrays and check for any remaining NaN
    X_train_array = X_train.values
    X_test_array = X_test.values
    
    if np.isnan(X_train_array).any():
        # Replace any remaining NaN with 0
        X_train_array = np.nan_to_num(X_train_array, nan=0.0)
    
    if np.isnan(X_test_array).any():
        # Replace any remaining NaN with 0
        X_test_array = np.nan_to_num(X_test_array, nan=0.0)
    
    y_train = train_data[target_col].values
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_array)
    X_test_scaled = scaler.transform(X_test_array)
    
    # Final check for NaN after scaling (shouldn't happen but just in case)
    if np.isnan(X_train_scaled).any():
        X_train_scaled = np.nan_to_num(X_train_scaled, nan=0.0)
    if np.isnan(X_test_scaled).any():
        X_test_scaled = np.nan_to_num(X_test_scaled, nan=0.0)
    
    # Train linear regression model
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)
    
    # Regression Analysis on Training Data
    y_train_pred = model.predict(X_train_scaled)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Adjusted R-squared
    n_train = len(y_train)
    p = len(valid_features)  # number of features (after filtering)
    adj_r2_train = 1 - (1 - train_r2) * (n_train - 1) / (n_train - p - 1) if (n_train - p - 1) > 0 else train_r2
    
    # Get regression coefficients
    coefficients = model.coef_
    intercept = model.intercept_
    
    # Feature importance (absolute value of coefficients)
    # Use valid_features since we may have filtered out some columns
    feature_importance = pd.DataFrame({
        'feature': valid_features,
        'coefficient': coefficients,
        'abs_coefficient': np.abs(coefficients)
    }).sort_values('abs_coefficient', ascending=False)
    
    # Predict missing values
    predicted = model.predict(X_test_scaled)
    
    # Get true values for evaluation
    # Use the stored true values (from before we made them missing)
    true_vals_list = []
    valid_indices = []
    
    for i, idx in enumerate(test_missing.index):
        if idx in true_values.index:
            true_vals_list.append(true_values.loc[idx])
            valid_indices.append(i)
        else:
            # Fallback: try to get from original data (shouldn't happen but just in case)
            val = true_values.get(idx, None)
            if val is not None and pd.notna(val):
                true_vals_list.append(val)
                valid_indices.append(i)
    
    # Convert to numpy array
    true_vals = np.array(true_vals_list)
    predicted = predicted[valid_indices]
    
    if len(true_vals) == 0 or len(predicted) == 0:
        return None, None, None
    
    # Calculate metrics
    if len(predicted) > 0 and len(true_vals) > 0:
        mse = mean_squared_error(true_vals, predicted)
        mae = mean_absolute_error(true_vals, predicted)
        test_r2 = r2_score(true_vals, predicted)
        
        # Regression statistics
        regression_stats = {
            'train_r2': train_r2,
            'train_adj_r2': adj_r2_train,
            'test_r2': test_r2,
            'coefficients': feature_importance,
            'intercept': intercept,
            'n_features': len(valid_features),
            'n_train': n_train,
            'n_test': len(true_vals)
        }
        
        return mse, mae, regression_stats
    
    return None, None, None

# ============================================================================
# REGRESSION ANALYSIS & IMPUTATION TEST
# ============================================================================
print("\n" + "="*80)
print("REGRESSION ANALYSIS & IMPUTATION TEST RESULTS")
print("="*80)

# Select a few target columns to test (columns with reasonable data)
target_columns = []
for col in numeric_cols[:10]:  # Test first 10 numeric columns
    non_missing = merged[col].notna().sum()
    if non_missing >= 20:  # At least 20 non-missing values
        target_columns.append(col)

if len(target_columns) == 0:
    # Fallback: use columns that exist
    target_columns = numeric_cols[:5]

print(f"\nTesting regression imputation on columns: {target_columns[:5]}")

results_all = {}
regression_analyses = {}

for target_col in target_columns[:5]:  # Test up to 5 columns
    print(f"\n{'='*80}")
    print(f"Target Column: {target_col}")
    print(f"{'='*80}")
    
    # Correlation-based imputation
    corr_mse_list = []
    corr_mae_list = []
    
    # Regression-based imputation
    reg_mse_list = []
    reg_mae_list = []
    train_r2_list = []
    test_r2_list = []
    adj_r2_list = []
    all_coefficients = []
    all_regression_stats = []
    
    # Run 10 iterations for both methods
    for i in range(10):
        # Correlation-based imputation
        corr_mse, corr_mae = impute_correlation(test_data, target_col, test_ratio=0.3, random_state=i)
        if corr_mse is not None and corr_mae is not None:
            corr_mse_list.append(corr_mse)
            corr_mae_list.append(corr_mae)
        
        # Regression-based imputation
        mse, mae, reg_stats = impute_and_evaluate(test_data, target_col, test_ratio=0.3, random_state=i)
        if mse is not None and mae is not None and reg_stats is not None:
            reg_mse_list.append(mse)
            reg_mae_list.append(mae)
            train_r2_list.append(reg_stats['train_r2'])
            test_r2_list.append(reg_stats['test_r2'])
            adj_r2_list.append(reg_stats['train_adj_r2'])
            all_coefficients.append(reg_stats['coefficients'])
            all_regression_stats.append(reg_stats)
    
    # Correlation-based imputation results
    if len(corr_mse_list) > 0:
        corr_mean_mse = np.mean(corr_mse_list)
        corr_mean_mae = np.mean(corr_mae_list)
        corr_std_mse = np.std(corr_mse_list)
        corr_std_mae = np.std(corr_mae_list)
        
        print(f"\n📊 CORRELATION-BASED IMPUTATION RESULTS ({len(corr_mse_list)} successful runs):")
        print("-" * 80)
        print(f"Imputation Accuracy:")
        print(f"  • Mean MSE:       {corr_mean_mse:.4f} (±{corr_std_mse:.4f})")
        print(f"  • Mean MAE:       {corr_mean_mae:.4f} (±{corr_std_mae:.4f})")
    else:
        corr_mean_mse = None
        corr_mean_mae = None
        print(f"\n❌ Correlation-based imputation failed for {target_col}")
    
    # Regression-based imputation results
    if len(reg_mse_list) > 0:
        reg_mean_mse = np.mean(reg_mse_list)
        reg_mean_mae = np.mean(reg_mae_list)
        reg_std_mse = np.std(reg_mse_list)
        reg_std_mae = np.std(reg_mae_list)
        mean_train_r2 = np.mean(train_r2_list)
        mean_test_r2 = np.mean(test_r2_list)
        mean_adj_r2 = np.mean(adj_r2_list)
        
        results_all[target_col] = {
            'corr_mean_mse': corr_mean_mse,
            'corr_mean_mae': corr_mean_mae,
            'reg_mean_mse': reg_mean_mse,
            'reg_mean_mae': reg_mean_mae,
            'reg_std_mse': reg_std_mse,
            'reg_std_mae': reg_std_mae,
            'mean_train_r2': mean_train_r2,
            'mean_test_r2': mean_test_r2,
            'mean_adj_r2': mean_adj_r2,
            'n_runs': len(reg_mse_list)
        }
        
        # Average feature coefficients across all runs
        if len(all_coefficients) > 0:
            avg_coefs = pd.concat(all_coefficients).groupby('feature').agg({
                'coefficient': 'mean',
                'abs_coefficient': 'mean'
            }).sort_values('abs_coefficient', ascending=False).reset_index()
        
        regression_analyses[target_col] = {
            'avg_coefficients': avg_coefs,
            'sample_stats': all_regression_stats[0]  # Use first run as sample
        }
        
        print(f"\n📊 REGRESSION-BASED IMPUTATION RESULTS ({len(reg_mse_list)} successful runs):")
        print("-" * 80)
        print(f"Model Performance:")
        print(f"  • Training R²:    {mean_train_r2:.4f} (±{np.std(train_r2_list):.4f})")
        print(f"  • Adjusted R²:    {mean_adj_r2:.4f} (±{np.std(adj_r2_list):.4f})")
        print(f"  • Test R²:        {mean_test_r2:.4f} (±{np.std(test_r2_list):.4f})")
        print(f"\nImputation Accuracy:")
        print(f"  • Mean MSE:       {reg_mean_mse:.4f} (±{reg_std_mse:.4f})")
        print(f"  • Mean MAE:       {reg_mean_mae:.4f} (±{reg_std_mae:.4f})")
        
        # Display top features (coefficients)
        print(f"\n🔍 Top 5 Most Important Features (by absolute coefficient):")
        print("-" * 80)
        top_features = avg_coefs.head(5)
        for idx, row in top_features.iterrows():
            print(f"  {idx+1}. {row['feature']:25s} | Coef: {row['coefficient']:10.4f} | |Coef|: {row['abs_coefficient']:8.4f}")
        
        # Show regression equation info
        sample = all_regression_stats[0]
        print(f"\n📐 Regression Model Info:")
        print(f"  • Number of features: {sample['n_features']}")
        print(f"  • Training samples:   {sample['n_train']}")
        print(f"  • Test samples:      {sample['n_test']}")
        print(f"  • Intercept:         {sample['intercept']:.4f}")
    else:
        print(f"  ❌ Could not perform regression imputation (insufficient data)")

# Summary
print("\n" + "="*80)
print("SUMMARY - Mean Average MSE and MAE across all columns (10 iterations)")
print("="*80)

if len(results_all) > 0:
    # Correlation-based imputation results
    all_corr_mse = [r['corr_mean_mse'] for r in results_all.values() if r['corr_mean_mse'] is not None]
    all_corr_mae = [r['corr_mean_mae'] for r in results_all.values() if r['corr_mean_mae'] is not None]
    
    # Regression-based imputation results
    all_reg_mse = [r['reg_mean_mse'] for r in results_all.values()]
    all_reg_mae = [r['reg_mean_mae'] for r in results_all.values()]
    all_train_r2 = [r['mean_train_r2'] for r in results_all.values()]
    all_test_r2 = [r['mean_test_r2'] for r in results_all.values()]
    
    overall_corr_mse = np.mean(all_corr_mse) if len(all_corr_mse) > 0 else None
    overall_corr_mae = np.mean(all_corr_mae) if len(all_corr_mae) > 0 else None
    overall_reg_mse = np.mean(all_reg_mse)
    overall_reg_mae = np.mean(all_reg_mae)
    overall_mean_train_r2 = np.mean(all_train_r2)
    overall_mean_test_r2 = np.mean(all_test_r2)
    
    print(f"\n📈 Overall Regression Performance:")
    print(f"  • Mean Training R²: {overall_mean_train_r2:.4f}")
    print(f"  • Mean Test R²:     {overall_mean_test_r2:.4f}")
    
    print("\n📋 Detailed results by column:")
    print("-" * 100)
    print(f"{'Column':<20s} | {'Train R²':<10s} | {'Test R²':<10s} | {'Reg MSE':<12s} | {'Reg MAE':<12s} | {'Corr MSE':<12s} | {'Corr MAE':<12s}")
    print("-" * 100)
    for col, res in results_all.items():
        corr_mse_str = f"{res['corr_mean_mse']:>11.4f}" if res['corr_mean_mse'] is not None else "N/A"
        corr_mae_str = f"{res['corr_mean_mae']:>11.4f}" if res['corr_mean_mae'] is not None else "N/A"
        print(f"{col:<20s} | {res['mean_train_r2']:>9.4f} | {res['mean_test_r2']:>9.4f} | "
              f"{res['reg_mean_mse']:>11.4f} | {res['reg_mean_mae']:>11.4f} | {corr_mse_str:>12s} | {corr_mae_str:>12s}")
    
    # Final summary text for MSE and MAE
    print("\n" + "="*80)
    print("FINAL RESULTS SUMMARY")
    print("="*80)
    print("\n🔗 CORRELATION-BASED IMPUTATION:")
    if overall_corr_mse is not None:
        print(f"  • Mean Squared Error (MSE): {overall_corr_mse:.4f}")
        print(f"  • Mean Absolute Error (MAE): {overall_corr_mae:.4f}")
    else:
        print("  • Correlation-based imputation was not successful for all columns")
    
    print("\n📊 REGRESSION-BASED IMPUTATION:")
    print(f"  • Mean Squared Error (MSE): {overall_reg_mse:.4f}")
    print(f"  • Mean Absolute Error (MAE): {overall_reg_mae:.4f}")
    print("\n" + "="*80)
else:
    print("\n❌ No successful imputation runs completed.")
    print("Please check if there's sufficient data in the merged dataset.")


Loaded datasets:
Air Quality: (140, 14)
Climate: (29363, 7)
CO2: (9466, 4)
Pollution: (487, 4)

Merged dataset shape: (98, 33)

Missing values in merged dataset:
BC                  96
NOX                 93
NO                  93
RELATIVEHUMIDITY    82
UM003               82
PM1                 81
TEMPERATURE         70
CO                  55
O3                  52
NO2                 49
SO2                 49
PM10                36
temp_max             1
PM2.5                1
temp_min             1
avg                  0
jan                  0
temp_mean            0
R1                   0
country_code         0
dec                  0
oct                  0
nov                  0
sep                  0
aug                  0
apr                  0
jul                  0
jun                  0
may                  0
feb                  0
mar                  0
co2_mean             0
co2_per_capita       0
dtype: int64

CORRELATION ANALYSIS

Correlation matrix shape: (32, 32)

Top cor

## Imputation 2

In [70]:
co2 = pd.read_csv('data/co2.csv')
aq = pd.read_csv('data/air_quality.csv')
pollution = pd.read_csv('data/pollution.csv')


In [71]:
aq.head()

,avg,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,country_code
0,19.000000,20.0,21.5,21.0,18.500000,16.000000,16.500000,14.500000,14.500000,20.000000,20.5,113.0,114.0,AFG
1,14.000000,20.0,23.0,13.0,10.000000,11.000000,10.000000,12.000000,11.000000,12.000000,16.0,11.0,20.0,ALB
2,23.000000,21.0,24.0,20.0,20.000000,15.000000,19.000000,27.000000,20.000000,28.000000,26.0,22.0,37.0,DZA
3,18.666667,17.0,20.0,19.0,19.666667,21.666667,21.333333,23.666667,21.000000,17.666667,16.0,14.0,14.0,AND
4,14.400000,27.6,31.8,22.3,32.800000,41.600000,17.266667,12.566667,13.166667,13.133333,12.4,15.8,5.4,AGO


In [72]:
pollution['pollutant'].value_counts()

pollutant
PM2.5               116
PM10                 71
NO2                  55
SO2                  55
O3                   52
CO                   49
TEMPERATURE          28
PM1                  17
RELATIVEHUMIDITY     16
UM003                16
NO                    5
NOX                   5
BC                    2
Name: count, dtype: int64

In [73]:
# Only using top pollutants
pollution = pollution[pollution['pollutant'].isin(['PM2.5', 'PM10', 'NO2', 'SO2', 'O3', 'CO'])]

In [74]:
co2.set_index(['country_code'], inplace=True)
co2 = co2[~co2.index.get_level_values('country_code').duplicated(keep='last')]


In [75]:
co2.head()

,year,value,co2_per_capita
country_code,,,
ABW,2016,883.747000,0.008439
AFG,2019,6079.999924,0.000160
AGO,2019,25209.999084,0.000779
ALB,2019,4829.999924,0.001683
AND,2019,500.000000,0.006535


In [76]:
aq.set_index(['country_code'], inplace=True)

In [77]:
co2['aq'] = aq['avg']

In [78]:
co2.head()

,year,value,co2_per_capita,aq
country_code,,,,
ABW,2016,883.747000,0.008439,NaN
AFG,2019,6079.999924,0.000160,19.000000
AGO,2019,25209.999084,0.000779,14.400000
ALB,2019,4829.999924,0.001683,14.000000
AND,2019,500.000000,0.006535,18.666667


In [79]:
# Suppose your DataFrame has columns: 'country_code', 'year', 'pollutant', 'unit', 'value'

pollution_pivot = pollution.pivot_table(
    index=['country_code'],   # Rows: one per country per year
    columns='pollutant',              # Columns: one per pollutant
    values='value'                    # Fill values from the 'value' column
).reset_index()

# Optional: flatten the column MultiIndex if needed
pollution_pivot.columns.name = None


In [80]:
pollution_pivot.head()

,country_code,CO,NO2,O3,PM10,PM2.5,SO2
0,AFG,NaN,NaN,NaN,NaN,-431.500000,NaN
1,AND,100.000000,4.700000,68.333333,11.000000,7.900000,0.200000
2,ARE,306.666667,9.185556,48.202308,247.722222,-204.200000,9.108889
3,ARG,590.000000,12.500000,26.000000,10.400000,6.148884,5.710000
4,ARM,NaN,NaN,NaN,NaN,10.400000,NaN


In [81]:
pollution_pivot.set_index(['country_code'], inplace=True)

In [82]:
for pollutant in pollution_pivot.columns:
    co2[pollutant] = pollution_pivot[pollutant]

In [83]:
co2.head()

,year,value,co2_per_capita,aq,CO,NO2,O3,PM10,PM2.5,SO2
country_code,,,,,,,,,,
ABW,2016,883.747000,0.008439,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,2019,6079.999924,0.000160,19.000000,NaN,NaN,NaN,NaN,-431.5,NaN
AGO,2019,25209.999084,0.000779,14.400000,NaN,NaN,NaN,NaN,NaN,NaN
ALB,2019,4829.999924,0.001683,14.000000,NaN,NaN,NaN,NaN,NaN,NaN
AND,2019,500.000000,0.006535,18.666667,100.0,4.7,68.333333,11.0,7.9,0.2


In [ ]:
co2.drop(columns=['year', 'value'], inplace=True)

In [98]:
co2.isna().sum() / len(co2)

co2_per_capita    0.000000
aq                0.359223
CO                0.762136
NO2               0.733010
O3                0.747573
PM10              0.665049
PM2.5             0.456311
SO2               0.733010
dtype: float64

In [99]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.metrics import mean_absolute_percentage_error

df = co2.copy()

def impute_and_evaluate_models(df, mask_fraction=0.4, random_state=42):
    np.random.seed(random_state)
    results = {}

    for col in df.columns:
        if df[col].isna().sum() == 0:
            continue

        known_idx = df[df[col].notna()].index
        mask_size = int(len(known_idx) * mask_fraction)
        mask_idx = np.random.choice(known_idx, size=mask_size, replace=False)
        true_values = df.loc[mask_idx, col].copy()
        df.loc[mask_idx, col] = np.nan

        predictors = df.columns[df.columns != col]

        # 1. Linear Regression
        X_train = df.loc[df[col].notna(), predictors].fillna(0)
        y_train = df.loc[df[col].notna(), col]
        X_pred = df.loc[df[col].isna(), predictors].fillna(0)
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        df.loc[df[col].isna(), col] = lr.predict(X_pred)
        mape_lr = mean_absolute_percentage_error(true_values, df.loc[mask_idx, col])

        # 2. KNN Imputer
        knn_imputer = KNNImputer(n_neighbors=3)
        df_knn = df.copy()
        df_knn[:] = knn_imputer.fit_transform(df_knn)
        mape_knn = mean_absolute_percentage_error(true_values, df_knn.loc[mask_idx, col])

        # 3. Iterative Imputer
        iter_imputer = IterativeImputer(random_state=random_state)
        df_iter = df.copy()
        df_iter[:] = iter_imputer.fit_transform(df_iter)
        mape_iter = mean_absolute_percentage_error(true_values, df_iter.loc[mask_idx, col])

        results[col] = {
            "LinearRegression": mape_lr,
            "KNNImputer": mape_knn,
            "IterativeImputer": mape_iter
        }

    return results

mape_results = impute_and_evaluate_models(df)
print(pd.DataFrame(mape_results).T )


       LinearRegression  KNNImputer  IterativeImputer
aq             1.579789    1.579789          1.579789
CO             2.133414    2.133414          2.133414
NO2            1.270085    1.270085          1.270085
O3             0.531262    0.531262          0.531262
PM10           3.684628    3.684628          3.684628
PM2.5          0.327614    0.327614          0.327614
SO2            6.554661    6.554661          6.554661
